In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -r requirements/build.txt
!pip install -v -e .
!pip install mmdet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

with open("/content/mmdetection/data/dataset/label_test.json") as f:
  data_train = json.load(f)

data_train['categories'] = [{'id': 1, 'name': 'Normal'}]

print(data_train.keys())

with open('/content/mmdetection/data/dataset/test.json', 'w') as f:
    json.dump(data_train, f)

dict_keys(['images', 'categories'])


In [ ]:
%cd /content/mmdetection
%mkdir checkpoint
%cd /content/mmdetection/checkpoint
!wget https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_x101_64x4d_fpn_mstrain-poly_3x_coco/mask_rcnn_x101_64x4d_fpn_mstrain-poly_3x_coco_20210526_120447-c376f129.pth

In [ ]:
!pip install roboflow
%cd /content/drive/MyDrive/colab/com

from roboflow import Roboflow
rf = Roboflow(api_key="API-KEY")
project = rf.workspace("ID").project("PJ_NAME)
dataset = project.version(11).download("coco-segmentation")

In [ ]:
%cd /content/drive/MyDrive/colab/com/final_dataset

import shutil

file_source = 'train/'
file_destination = '.'

shutil.move(file_source + '_annotations.coco.json', file_destination)

/content/drive/MyDrive/colab/lg_com/final_dataset


'./_annotations.coco.json'

In [ ]:
import json

with open("_annotations.coco.json") as f:
  data_test = json.load(f)

data_test['categories'] = [{'id': 1, 'name': 'Normal'}]

print(data_test.keys())

with open('_annotations.coco.json', 'w') as f:
    json.dump(data_test, f)

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])


In [ ]:
import json

with open("_annotations.coco.json") as f:
  data_test = json.load(f)

data_test['categories']

[{'id': 1, 'name': 'Normal'}]

In [ ]:
# custom/bora_mask.py
_base_ = '/content/mmdetection/configs/convnext/cascade_mask_rcnn_convnext-s_p4_w7_fpn_giou_4conv1f_fp16_ms-crop_3x_coco.py'
 
image_size = (1024, 1280)
dataset_type = 'CocoDataset'

# 데이터 폴더 설정
data_root = '/content/drive/MyDrive/colab/com/final_dataset/'
classes = ('Normal',)

# 데이터 설정
data = dict(
    samples_per_gpu=1,
    workers_per_gpu=1,
    train=dict(
      img_prefix=data_root + "train/",
      classes = classes,
      ann_file=data_root + "_annotations.coco.json"
),
    val=dict(
        img_prefix=data_root + "train/",
        classes = classes,
        ann_file=data_root + "_annotations.coco.json"
),
    test=dict(
        img_prefix=data_root + "test/",
        classes = classes,
        ann_file=data_root + "test.json"
)
)
# log 저장 위치
checkpoint_config = dict(interval=3,out_dir='/content/drive/MyDrive/colab/com/final_dataset/0805_convnext')

# 평가 방법
evaluation = dict(interval=3, metric=['bbox', 'segm'])

# 사전 가중치 사용
# https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_x101_64x4d_fpn_mstrain-poly_3x_coco/mask_rcnn_x101_64x4d_fpn_mstrain-poly_3x_coco_20210526_120447-c376f129.pth
# pretrained = '/content/mmdetection/checkpoint/mask_rcnn_x101_64x4d_fpn_mstrain-poly_3x_coco_20210526_120447-c376f129.pth'

# epoch 설정 
#runner = dict(type='IterBasedRunner', max_iters=5000)
runner = dict(type='EpochBasedRunner', max_epochs= 12)

# batch size 설정
auto_scale_lr = dict(enable=False, base_batch_size=2)

In [ ]:
import json

with open("/content/drive/MyDrive/InstanceSegmentTest/dataset/test.json") as f:
  data_train = json.load(f)

data_train['categories'] = [{'id': 1, 'name': 'Normal'}]

print(data_train.keys())

with open('/content/drive/MyDrive/InstanceSegmentTest/dataset/test.json', 'w') as f:
    json.dump(data_train, f)

dict_keys(['images', 'categories'])


In [ ]:
import os
os.chdir('/content/mmdetection/')

In [ ]:
!pip install mmcls

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578 kB 20.8 MB/s 


In [ ]:
!python ./tools/train.py /content/mmdetection/configs/custom/bora_mask.py --work-dir /content/drive/MyDrive/colab/com/final_dataset/0805_convnext

In [ ]:
#!python tools/test.py /content/mmdetection/configs/custom/bora_mask.py /content/drive/MyDrive/InstanceSegmentTest/dataset/'output_0801_(1)'/bora_0802/latest.pth --show-dir results/bora_mask --format-only --eval-options "jsonfile_prefix=./work_dirs/bora_cfg/"
!python tools/test.py /content/mmdetection/configs/custom/bora_mask.py /content/drive/MyDrive/InstanceSegmentTest/dataset/'output_0801_(1)'/bora_0802/latest.pth --format-only --eval-options "jsonfile_prefix=./work_dirs/bora_cfg/"

loading annotations into memory...
Done (t=0.54s)
creating index...
index created!
load checkpoint from local path: /content/drive/MyDrive/InstanceSegmentTest/dataset/output_0801_(1)/bora_0802/latest.pth
[                                                  ] 0/350, elapsed: 0s, ETA:/content/mmdetection/mmdet/models/utils/positional_encoding.py:81: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature**(2 * (dim_t // 2) / self.num_feats)
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/n

In [ ]:
from glob import glob 
import cv2
from google.colab.patches import cv2_imshow

data_list = glob('/content/mmdetection/results/bora_mask/*.jpg')
for i in data_list[:2]:
  src = cv2.imread(i)
  cv2_imshow(src)

In [ ]:
os.listdir('/content/mmdetection/work_dirs/bora_cfg/')

['.bbox.json', '.segm.json']

In [ ]:
import json

with open("/content/mmdetection/work_dirs/bora_cfg/.bbox.json") as f:
    out_data1 = json.load(f)

with open("/content/mmdetection/work_dirs/bora_cfg/.segm.json") as f:
    out_data2 = json.load(f)

In [ ]:
with open('/content/mmdetection/work_dirs/bora_cfg/predict.segm.json', 'w') as outfile:
    json.dump(out_data2, outfile, indent=4)

In [ ]:
############################################### end #########################